In [ ]:
import pandas as pd
import io
import os
from datetime import datetime
# AWS
import boto3

# Credenciais aws
aws_access_key_id = os.environ["AWS_ACCESS_KEY_ID"]
aws_secret_access_key = os.environ["AWS_SECRET_ACCESS_KEY"] 

# Bucket e Key (caminho) do arquivo no S3
BUCKET_NAME='CCCCCCCCCCCC0336967'



path_parquet = f'XXXXXXXXXXXXXXX/fat/year=2023/month=04/escritorio_venda_2023-04.parquet'


#Configura a sessão do client da aws
s3_resource = boto3.resource('s3',
    region_name = "us-east-1",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key)

buffer = io.BytesIO()
#Coletando dados da raw-zone e reconstruindo o json
obj_buffer = s3_resource.Object(BUCKET_NAME,path_parquet)

obj_buffer.download_fileobj(buffer)

df = pd.read_parquet(buffer)

df.head(10)

In [ ]:
mes = '03'
df['data'] = f'2023-{mes}-01'
df['ultima_atualizacao'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df = df.astype(str)
df.head(10)

In [ ]:
path_parquet = f'XXXXXXXXXXXXXXXXXXXXXX/escritorio_venda/fat/year=2023/month={mes}/escritorio_venda_2023-{mes}.parquet'

In [ ]:
s3_client = boto3.client(
    's3', 
    region_name='us-east-1',
    aws_access_key_id=aws_access_key_id, 
    aws_secret_access_key=aws_secret_access_key 
)
out_buffer = io.BytesIO() # criando espaço na memória
df.to_parquet(out_buffer, index=False) # salvando na memória em formato parquet

s3_client.put_object(Bucket=BUCKET_NAME, Key=path_parquet, Body=out_buffer.getvalue())

In [ ]:
manifest_contents_new = f's3://{BUCKET_NAME}/{path_parquet}\n'
key_manifest = f'CCCCCCCCCCCCCCCCCCCCCCC/escritorio_venda/fat/_symlink_format_manifest/manifest2023-{mes}'
print('manifest')
s3_client.put_object(
    Body=manifest_contents_new,
    Bucket=BUCKET_NAME,
    Key=key_manifest,
)

In [ ]:
key_manifest = ''

In [ ]:
manifest_obj = s3_client.get_object(
    Bucket=BUCKET_NAME,
    Key=key_manifest
)

manifest_contents = manifest_obj['Body'].read( ).decode('utf-8')

manifest_contents_new = f'{manifest_contents}s3://{BUCKET_NAME}/{path_parquet}\n'   
print(manifest_contents_new)

In [ ]:
s3_client.put_object(
    Body=manifest_contents_new,
    Bucket=BUCKET_NAME,
    Key=key_manifest,
)